# PSTA

In [1]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import math
from multiprocessing import  Pool

In [2]:
df = pd.read_csv('../../Semesteroppgave/datasets/dataset/dataset_twitter/tweetsreplies4.tsv', encoding='cp1252', sep="\t", usecols=['timestamp_ms','longitude', 'latitude', 'text'])

## Preprocessing
Get detailed locations and timestamp and tokenize the text.
* Fetch more location info from longitude and latitude using reverse_encoder https://github.com/thampiman/reverse-geocoder
* Convert ms timestamps to datetime object.
* Preprocess text using the preprocessor https://github.com/s/preprocessor and remove stopwords using gensim.

In [3]:
coordinates = list(df[['latitude','longitude']].itertuples(index=False, name=None))
locations = rg.search(coordinates)

Loading formatted geocoded file...


In [4]:
locations_df = pd.json_normalize(locations)[['name', 'admin1', 'admin2', 'cc']]

In [5]:
# df['timestamp'] = pd.to_datetime(df['timestamp_ms'], unit='ms') #.dt.date

In [6]:
df = pd.concat([df, locations_df], axis=1)

In [7]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [8]:
# This takes a very long time, wonder if it hangs or something? It is the clean-command that is slow.
# It is always the same tweet that returns when I force stop it.
def add_features(df):
    df['text'] = df['text'].apply(p.clean)
    return df
df = parallelize_dataframe(df.head(100), add_features)

In [9]:
# make lower case and remove white space, must be done after cleaning
df['text']  = df['text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

In [10]:
# remove stopwords, must be done after cleaning and removal of white space
# my_func = lambda x: remove_stopwords(x['doc'])
def remove_stopwords_f(df):
    df['text'] = df['text'].apply(remove_stopwords)
    return df
df = parallelize_dataframe(df, remove_stopwords_f)

In [11]:
df = df.drop(df[df.text == ''].index)

In [12]:
#df = df.groupby(['timestamp_ms', 'admin1']).agg(list)

In [15]:
df = df.sort_values(by=['timestamp_ms', 'admin1'])

In [16]:
df

,timestamp_ms,longitude,latitude,text,name,admin1,admin2,cc
0,1443887012127,-3.703508,40.477795,el bara lleva penaltis favor y el madrid slo v...,Tetuan de las Victorias,Madrid,Provincia de Madrid,ES
3,1443887056044,-5.923545,37.383117,envidia,Sevilla,Andalusia,Provincia de Sevilla,ES
7,1443887073903,-66.578926,6.422820,acuerdo pero otra cosa es que ellos lo sepan c...,Puerto Carreno,Vichada,,CO
5,1443887110156,-0.352529,39.422484,tu eres retrasado,Alfafar,Valencia,Provincia de Valencia,ES
13,1443887259772,20.951529,52.230058,oblaem si herbat,Ochota,Masovian Voivodeship,Warszawa,PL
...,...,...,...,...,...,...,...,...
50,1443978210545,-7.925291,40.689216,qnd devias tar estudar e vais tirar fotos basi...,Abraveses,Viseu,Viseu,PT
43,1443988487058,-7.925291,40.689216,pq e que eu antes nunca sorria com dentes e ag...,Abraveses,Viseu,Viseu,PT
35,1444075868135,-66.578926,6.422820,den rt,Puerto Carreno,Vichada,,CO
8,1444247467474,-99.153366,19.432836,el madrid lleva champions regaladas barcelona ...,Cuauhtemoc,Mexico City,,MX
